In [15]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import ApproxTimeEvolution

import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold

import random
from imblearn.under_sampling import RandomUnderSampler

In [16]:
# function to load and label datasets
def load_data(file_num):
    # File paths
    train_neg_path = f'./data/Hemo40D/data{file_num}/train/neg.fa_encod.csv'
    train_pos_path = f'./data/Hemo40D/data{file_num}/train/pos.fa_encod.csv'
    test_neg_path = f'./data/Hemo40D/data{file_num}/test/neg.fa_encod.csv'
    test_pos_path = f'./data/Hemo40D/data{file_num}/test/pos.fa_encod.csv'
    
    # Load the CSV files
    train_neg = pd.read_csv(train_neg_path)
    train_pos = pd.read_csv(train_pos_path)
    test_neg = pd.read_csv(test_neg_path)
    test_pos = pd.read_csv(test_pos_path)
    
    # Insert labels (0 for negative, 1 for positive)
    train_neg.insert(0, 'Label', 0)
    train_pos.insert(0, 'Label', 1)
    test_neg.insert(0, 'Label', 0)
    test_pos.insert(0, 'Label', 1)

    frames_trian = [train_neg,train_pos]
    frames_test  = [test_neg,test_pos]
    train_df     = pd.concat(frames_trian)
    test_df      = pd.concat(frames_test)

    return train_df, test_df

def pre_process(file_num):
    train_df, test_df = load_data(file_num)
    column_names = train_df.columns[3:].tolist()

    scaler        = StandardScaler()  # mean 0 and std 1
    undersampleer = RandomUnderSampler(random_state=42)

    train_df[column_names] = scaler.fit_transform(train_df[column_names])
    test_df[column_names]  = scaler.transform(test_df[column_names])
    
    x      = train_df[column_names].to_numpy()
    y      = train_df['Label'].to_numpy()
    x_test = test_df[column_names].to_numpy()
    y_test = test_df['Label'].to_numpy()

    X,y = undersampleer.fit_resample(x, y)
    return X,y,x_test,y_test


In [17]:
x_hemo1,y_hemo1,x_hemo1_test,y_hemo1_test = pre_process(1)
x_hemo2,y_hemo2,x_hemo2_test,y_hemo2_test = pre_process(2)
x_hemo3,y_hemo3,x_hemo3_test,y_hemo3_test = pre_process(3)

In [18]:
# Function to create one random Pauli String 
def one_operator(num_qubits):
    ops_list = [qml.PauliX, qml.PauliY, qml.PauliZ,qml.Identity] # Pauli matrices
    return qml.operation.Tensor(*(random.choice(ops_list)(i) for i in range(num_qubits)))

# Function to create multiple random Pauli String
def hamiltonian_operators(num_qubits, num_ops,num_samples=1):
    ops_all = []
    for _ in range(num_samples):
        ops = []
        for _ in range(num_ops):
            op = one_operator(num_qubits)
            ops.append(op)
        ops_all.append(ops)
    return ops_all


# Kernel matrix 
def kernel_matrix(A,B):
    A = np.array(A)
    B = np.array(B)
    
    return np.absolute(np.matmul(np.conjugate(A),B.T)**2)

#####################################################

random.seed(42)

num_qubits = [6,8,10,12,14]    # number of qubits
L          = 40                # number of operators = number of descriptors
n_sample   = 30                # number of samples for each number of qubits
ops_dict   = {}

for n in num_qubits:
    ops_dict[f'{n}_qubits'] = hamiltonian_operators(num_qubits=n,num_ops=L,num_samples=n_sample)


In [19]:
# function to train quantum kernels with StratifiedKFold
# for different number of qubits, time, trotter steps and Pauli Strings
def train(n_qubits,time,step,X,y,n_ops=3):
    print('{:5s} | {:5s} | {:3s} | {:3s} | {:7s}  | {:7s} | {:7s}'""
            .format('CV','Num Qubits','Time','Step','Ops Index','Train_Acc','Val_acc'))
    
    Column_name = ['Num Qubits','Time','Step','Ops Index','Training Accuracy','Validation Accuracy']
    kf = StratifiedKFold(n_splits=5, random_state=42, shuffle= True)

    results = []
    for n in n_qubits:
        dev = qml.device("lightning.qubit", wires=n)        
        for t in time:
            for s in step:
                for o in range(n_ops): 
                
                    ops = ops_dict[str(n)+'_qubits'][o] 

                    @qml.qnode(dev, interface="autograd")
                    def kernel(ops, x,time=1,steps=1):   

                        hamiltonian = qml.Hamiltonian(x, ops)
                        ApproxTimeEvolution(hamiltonian, time, steps)

                        return qml.state()
                    
                    for i,(train_index, val_index) in enumerate(kf.split(X, y)):
                        X_train, X_val = X[train_index], X[val_index]
                        y_train, y_val = y[train_index], y[val_index]

                        q_state    = [ kernel(ops,x,t,s) for x in X_train] # training states
                        val_state  = [ kernel(ops,x,t,s) for x in X_val] # validation states

                        k_matrix   = kernel_matrix(q_state,q_state) # Compute kernel matrix with training set
                        svm        = SVC(kernel='precomputed').fit(k_matrix, y_train)  # Fit 
                        train_pred = svm.predict(k_matrix)
                        train_acc  = accuracy_score(y_train,train_pred)

                        val_matrix = kernel_matrix(q_state,val_state) # Compute kernel matrix with validation set
                        y_pred     = svm.predict(val_matrix.T) # Prediction
                        val_acc    = accuracy_score(y_val,y_pred) # Validation accuracy
                        result = [n,t,s,o,train_acc,val_acc]
                        results.append(result)

                        print("{:>4}/5| {:>11}| {:>4} | {:>4} | {:>11d}| {:>10f}| {:7f}"
                                "".format(i+1,n,t,s,o,train_acc,val_acc))


    result_df = pd.DataFrame(results,columns=Column_name)


    return result_df

In [20]:
# Function to compute test accuracy on quantum kernels
def testing(num_qubits,time,step,ops_index,xdata,ydata,x_test,y_test):

    n_qubits = num_qubits
    dev_kernel = qml.device("lightning.qubit", wires=n_qubits)

    @qml.qnode(dev_kernel, interface="autograd")
    def kernel(x,ops,time=1,steps=1):   
        
        hamiltonian_1 = qml.Hamiltonian(x, ops)
        ApproxTimeEvolution(hamiltonian_1, time, steps)
        return qml.state()
    
    np.random.seed(42)
    

    ops_n8 = ops_dict[str(n_qubits)+'_qubits'][ops_index]

    q_state    = [ kernel(x,ops_n8,time,step) for x in xdata] # training states
    test_state = [ kernel(x,ops_n8,time,step) for x in x_test] # testing states

    k_matrix   = kernel_matrix(q_state,q_state) # Compute kernel matrix with training set
    svm        = SVC(kernel='precomputed').fit(k_matrix, ydata)  # Fit 

    test_matrix = kernel_matrix(q_state,test_state)
    test_pred   = svm.predict(test_matrix.T)
    test_score  = svm.decision_function(test_matrix.T)

    test_acc    = accuracy_score(y_test,test_pred)

    precision = precision_score(y_test, test_pred,pos_label=1)
    recall    = recall_score(y_test, test_pred,pos_label=1)
    f1_       = f1_score(y_test, test_pred,pos_label=1)
    roc_auc_  = roc_auc_score(y_test, test_score)

    print(f"Testing accuracy: {test_acc:.5f}")
    print(f"Precision: {precision:.5f}")
    print(f"Recall: {recall:.5f}")
    print(f"F1: {f1_:.5f}")
    print(f"ROC AUC: {roc_auc_:.5f}")


    return [test_acc,precision,recall,f1_,roc_auc_]

In [47]:
def df_avg(df_TrainVal,df_test):
    df_TrainVal_mean = df_TrainVal.groupby(['Num Qubits', 'Time','Step','Ops Index']).agg(
                                            Average_Train_Accuracy=('Training Accuracy', 'mean'),
                                            Average_Val_Accuracy=('Validation Accuracy', 'mean')).reset_index()
    df_TrainVal_mean['Test_Accuracy'] =  df_test

    return df_TrainVal_mean

### Grid search
- First grid search with wider range of time steps.
- Second grid search with finer area where the gap between train and validation accuracy is small.

In [ ]:
n=[6]
time = [0.1,0.3,0.5,0.7,0.9,1]
step = [1,5,10,20]
hemo1_wide_range = train(n,time,step,x_hemo1,y_hemo1)
hemo1_wide_range.to_csv('./result/hemo1_wide_range.csv',index=False)

In [ ]:
n=[6]
time = [0.12,0.15,0.17,0.2,0.22,0.25,0.27]
step=[1,5,10,20]
hemo1_fine_search = train(n,time,step,x_hemo1,y_hemo1)
hemo1_fine_search.to_csv('./result/hemo1_fine_search.csv',index=False)

In [ ]:
n=[6]
time = [0.1,0.3,0.5,0.7,0.9,1]
step = [1,5,10]
hemo2_wide_range = train(n,time,step,x_hemo2,y_hemo2)
hemo2_wide_range.to_csv('./result/hemo2_wide_range.csv',index=False)

In [ ]:
n=[6]
time = [0.12,0.15,0.17,0.2,0.22,0.25,0.27]
step=[1,5,10,20]
hemo2_fine_search = train(n,time,step,x_hemo2,y_hemo2)
hemo2_fine_search.to_csv('./result/hemo2_fine_search.csv',index=False)

In [ ]:
n=[6]
time = [0.1,0.3,0.5,0.7,0.9,1]
step = [1,5,10,20]
hemo3_wide_range = train(n,time,step,x_hemo3,y_hemo3)
hemo3_wide_range.to_csv('./result/hemo3_wide_range.csv',index=False)

In [ ]:
n=[6]
time = [0.12,0.15,0.17,0.2,0.22,0.25,0.27]
step=[1,5,10,20]
hemo3_fine_search = train(n,time,step,x_hemo3,y_hemo3)
hemo3_fine_search.to_csv('./result/hemo3_fine_search.csv',index=False)

---------------------
#### Train, validation and testing ACC on 30 sets of random Pauli Strings with best $t$ and $s$ identified from grid search

In [ ]:
n = [6]
time = [0.3]
step = [10]

hemo1_30sets = train(n,time,step,x_hemo1,y_hemo1,n_ops=30)
hemo1_30sets_test = [testing(n,time,step,i,x_hemo1,y_hemo1,x_hemo1_test,y_hemo1_test)[0] for i in range(30)] # Collect only test accuracy

hemo1_30sets_mean  = df_avg(hemo1_30sets,hemo1_30sets_test)
hemo1_30sets_mean.to_csv('./result/hemo1_30sets_mean.csv',index=False)

In [ ]:
n = [6]
time = [0.15]
step = [10]

hemo2_30sets = train(n,time,step,x_hemo2,y_hemo2,n_ops=30)
hemo2_30sets_test = [testing(n,time,step,i,x_hemo2,y_hemo2,x_hemo2_test,y_hemo2_test)[0] for i in range(30)]

hemo2_30sets_mean  = df_avg(hemo2_30sets,hemo2_30sets_test)
hemo2_30sets_mean.to_csv('./result/hemo2_30sets.csv',index=False)

In [ ]:
n = [6]
time = [0.15]
step = [10]
hemo3_30sets = train(n,time,step,x_hemo3,y_hemo3,n_ops=30)
hemo3_30sets_test = [testing(n,time,step,i,x_hemo3,y_hemo3,x_hemo3_test,y_hemo3_test)[0] for i in range(30)]

hemo3_30sets_mean  = df_avg(hemo3_30sets,hemo3_30sets_test)
hemo3_30sets_mean.to_csv('./result/hemo3_30sets.csv',index=False)